# OpenStack API

El API de OpenStack permite la gestión de todos los componentes de la infraestructura.

El API de OpenStack no existe. Existe el API de cada componente de OpenStack.

Vamos a realizar las siguientes operaciones como ejemplo de uso:
* keystone:
  * autenticación
* neutron
  * creación de red
  * creación de router
* nova
  * creación de máquina virtual
  * obtención de consola
  

## Variables de entorno
Inicializamos las variables de entorno tomando como partida el archivo de openrc.sh que nos bajamos desde la interfaz de OpenStack:

In [ ]:
import sys
sys.path.append('/usr/lib/python2.7/dist-packages/')

config={}
config["OS_AUTH_URL"]="http://openstack-vcenter:5000/v3"
config["OS_PROJECT_ID"]="9b0989dfbdbd4d048f88720d4ccbf9ac"
config["OS_PROJECT_NAME"]="admin"
config["OS_USER_DOMAIN_NAME"]="default"
config["OS_USERNAME"]="admin"
config["OS_PASSWORD"]="admin"
config["OS_REGION_NAME"]="RegionOne"
config["name"]="osl-computer"

## Autenticación
Enviamos nuestras credenciales y obtenemos un token en las cabeceras ( http://developer.openstack.org/api-ref/identity/v3/index.html?expanded=password-authentication-with-scoped-authorization-detail ):

### Preparación de cabeceras

In [10]:
# Peticiones http
import requests
# Tratamiento de json
import json

headers = {}
headers["Content-Type"] = 'application/json'

### Petición de token

In [11]:
data = """
{
    "auth": {
        "identity": {
            "methods": [
                "password"
            ],
            "password": {
                "user": {
                    "name": "%s",
                    "password": "%s",
                    "domain": {
                        "name": "%s"
                    }
                }
            }
        },
        "scope": {
            "project": {
                "id": "%s",
                "domain": {
                    "name": "%s"
                }
            }
        }
    }
}
""" % (config["OS_USERNAME"],
       config["OS_PASSWORD"],
       config["OS_USER_DOMAIN_NAME"],
       config["OS_PROJECT_ID"],
       config["OS_USER_DOMAIN_NAME"])
#print data
headers["Content-Type"] = 'application/json'
#
r = requests.post(config["OS_AUTH_URL"] + "/auth/tokens",
                  data=data, headers=headers)
token = json.loads(r.text)
token_id = r.headers["X-Subject-Token"]
print ("token_id: " + str(token_id))
print ("token: " + str(json.dumps(token, indent=4)))




token_id: gAAAAABYeyhGVgrST1xFbaFR2KoTK6JFLJhrgAAlHn5kWrMs1Lf-Crt1I9D_OMdzzp5_gyub3CcIYh3oky09C3jodxcx2lB4joKYK8mecov8jcXNLsXS_ua3mD8jIdrB_rytKKHaUyCTYGFegz5s3g9TY5KWXgLNanYiy0wdJgmgryGq8MCHrYA
token: {
    "token": {
        "methods": [
            "password"
        ], 
        "roles": [
            {
                "id": "9ca495ac8ad846b5ae575f92cf86b3e6", 
                "name": "admin"
            }
        ], 
        "expires_at": "2017-01-15T08:44:06.454784Z", 
        "project": {
            "domain": {
                "id": "7b49c5bc915f46a489aae1fd776cf78c", 
                "name": "default"
            }, 
            "id": "9b0989dfbdbd4d048f88720d4ccbf9ac", 
            "name": "admin"
        }, 
        "catalog": [
            {
                "endpoints": [
                    {
                        "url": "http://openstack-vcenter:9292", 
                        "interface": "internal", 
                        "region": "RegionOne", 
                      

### Agregamos el token a las cabeceras

In [12]:
headers["X-Auth-Token"] = token_id
print json.dumps(headers, indent=4)

{
    "Content-Type": "application/json", 
    "X-Auth-Token": "gAAAAABYeyhGVgrST1xFbaFR2KoTK6JFLJhrgAAlHn5kWrMs1Lf-Crt1I9D_OMdzzp5_gyub3CcIYh3oky09C3jodxcx2lB4joKYK8mecov8jcXNLsXS_ua3mD8jIdrB_rytKKHaUyCTYGFegz5s3g9TY5KWXgLNanYiy0wdJgmgryGq8MCHrYA"
}


## Creación de red privada
La red de cada PC virtual debe ser independiente y aislada del resto:
* obtenemos la información de la plataforma para trabajar con ella
* guardamos red con salida pública
* creamos red y subred privada
* creamos puerto en subred privada
* creamos router en subred privada y pública
* asignamos puerto a router


### Obtenemos el endpoint para el servicio de red

In [ ]:
network_url = ""
endpoint_type = "network"
interface_type = "public"
for i in range(len(token["token"]["catalog"])):
    if (token["token"]["catalog"][i]["type"] == endpoint_type):
        for j in range(len(token["token"]["catalog"][i]["endpoints"])):
            if (token["token"]["catalog"][i]["endpoints"][j]["interface"] == interface_type):
                network_url = token["token"]["catalog"][i]["endpoints"][j]["url"]
print (network_url)

### Obtenemos el listado de redes actual
Necesitamos saber cual es nuestro entorno de red para crear más elementos ( http://developer.openstack.org/api-ref/networking/v2/index.html?expanded=list-networks-provider-network-detail#api-guide ):

In [ ]:
r = requests.get(network_url + "/v2.0/networks", headers=headers)
networks = json.loads(r.text)
print ((json.dumps(networks, indent=4)))

In [ ]:
# Obtenemos el network_id de la red publica
public_network = {}
for network in networks["networks"]:
    if network["router:external"]:
        public_network = network
print ((json.dumps(public_network, indent=4)))

In [ ]:
# Creamos la red de la instancia
private_net_name = config["name"] + "_net"
data = """
{
    "network": {
        "name": "%s",
        "admin_state_up": true
    }
}
""" % private_net_name
r = requests.post(network_url + "/v2.0/networks",
                  headers=headers, data=data)
private_net = json.loads(r.text)
print ((json.dumps(private_net, indent=4)))

In [ ]:
# Creamos la subred de la instancia
# subnetwork_url = "http://openstack.paradigmadigital.com:9696/v2.0/subnets"
private_subnet_name = config["name"] + "_subnet"
data = """
{
 "subnet": {
     "name": "%s",
     "ip_version": 4,
     "network_id": "%s",
     "cidr": "172.17.235.0/24",
     "gateway_ip": "172.17.235.1",
     "allocation_pools": [
         {
             "start": "172.17.235.10",
             "end": "172.17.235.100"
         }
     ],
     "enable_dhcp": "true"
 }
}
""" % (private_subnet_name, private_net["network"]["id"])
r = requests.post(network_url + "/v2.0/subnets", headers=headers, data=data)
private_subnet = json.loads(r.text)
print ((json.dumps(private_subnet, indent=4)))



In [ ]:
# Creamos un router para dar salida a la red publica hacia el exterior
# routers_url = "http://openstack-vcenter:9696/v2.0/routers"
router_name = config["name"] + "_router"
data = """
{
    "router": {
        "name": "%s",
        "external_gateway_info": {
            "network_id": "%s"
        }
    }
}
""" % (router_name, public_network["id"])
r = requests.post(network_url + "/v2.0/routers", headers=headers, data=data)
external_router = json.loads(r.text)
print ((json.dumps(external_router, indent=4)))

In [ ]:
# Conectamos el router público con la red privada
# add_router_interface_url = routers_url + "/" + external_router_id +
# "/add_router_interface"
data = """
{
    "subnet_id": "%s"
}
""" % private_subnet["subnet"]["id"]
r = requests.put(network_url + "/v2.0/routers/" + external_router["router"]["id"] + "/add_router_interface",
                 headers=headers, data=data)
external_router_connections = json.loads(r.text)
print ((json.dumps(external_router_connections, indent=4)))

In [ ]:
env = {}
env["name"] = config["name"]
env["network"] = {}
env["network"]["public"] = public_network
env["network"]["private_net"] = private_net["network"]
env["network"]["private_subnet"] = private_subnet["subnet"]
env["network"]["external_router"] = external_router["router"]
print ((json.dumps(env, indent=4)))

#config["net_id"] = private_net["network"]["id"]
#config["router_id"] = external_router["router"]["id"]
#config["subnet_id"] = private_subnet["subnet"]["id"]
#print ((json.dumps(config, indent=4)))


In [ ]:
# Obtenemos el endpoint para el servicio de compute
compute_url = ""
endpoint_type = "compute"
interface_type = "public"
for i in range(len(token["token"]["catalog"])):
    if (token["token"]["catalog"][i]["type"] == endpoint_type):
        for j in range(len(token["token"]["catalog"][i]["endpoints"])):
            if (token["token"]["catalog"][i]["endpoints"][j]["interface"] == interface_type):
                compute_url = token["token"]["catalog"][i]["endpoints"][j]["url"]
print ((compute_url))

In [ ]:
# Create computer
name = env["name"] + "_computer"
image = "eed561f3-0199-4242-92f8-3772a6c6d88d"
flavor = "2"
#print ((json.dumps(env, indent=4)))
data = """
{
    "server" : {
        "name" : "%s",
        "imageRef" : "%s",
        "flavorRef" : "%s",
        "availability_zone": "nova",
        "security_groups": [
            {
                "name": "default"
            }
        ],
        "networks": [
            {
                "uuid": "%s"
            }
        ]
    }
}
""" % (name, image, flavor, env["network"]["private_net"]["id"])
r = requests.post(compute_url + "/servers", headers=headers, data=data)
server_env = json.loads(r.text)
print ((json.dumps(server_env["server"], indent=4)))
env["server"] = server_env["server"]


In [ ]:
# Get console url
data = """
{
    "os-getVNCConsole": {
        "type": "novnc"
    }
}
"""
#data = """
#{
    #"os-getSPICEConsole": {
        #"type": "spice-html5"
    #}
#}
#""""
r = requests.post(compute_url + "/servers/" + env["server"]["id"] + "/action",
                  headers=headers, data=data)
print ((r.text))
console_env = json.loads(r.text)
print ((json.dumps(console_env, indent=4)))
